In [23]:
import geopandas as gpd

In [24]:
county_fips = '13059'
nsa_api_call = 'https://nsi.sec.usace.army.mil/nsiapi/structures?fips=' + county_fips
nsi_gdf = gpd.read_file(nsa_api_call)

In [25]:
nsi_gdf.head()

,fd_id,bid,occtype,st_damcat,bldgtype,found_type,cbfips,pop2amu65,pop2amo65,pop2pmu65,...,source,med_yr_blt,firmzone,o65disable,u65disable,x,y,ground_elv,ground_elv_m,geometry
0,508520377,865RXJGC+98F-4-4-4-3,RES1-2SNB,RES,W,S,130590301001007,1,0,1,...,X,2001,NaN,0.21,0.03,-83.379191,33.975939,663.993136,202.385101,POINT (-83.37919 33.97594)
1,508490030,865RWGVJ+GXV-48-76-38-78,COM4,COM,C,S,130591306001029,0,0,0,...,P,1988,NaN,0.21,0.03,-83.467581,33.943859,720.858109,219.717545,POINT (-83.46758 33.94386)
2,508490031,865RWGWP+66V-4-3-4-3,REL1,PUB,M,S,130591306001028,0,0,0,...,E,1988,NaN,0.21,0.03,-83.464462,33.945615,739.188881,225.304764,POINT (-83.46446 33.94561)
3,508490032,865RWGWP+66V-4-3-4-3,COM7,COM,M,S,130591306001028,0,0,1,...,E,1988,NaN,0.21,0.03,-83.464462,33.945615,739.188881,225.304764,POINT (-83.46446 33.94561)
4,508490033,865RWGWP+66V-4-3-4-3,COM4,COM,M,S,130591306001028,0,0,1,...,E,1988,NaN,0.21,0.03,-83.464462,33.945615,739.188881,225.304764,POINT (-83.46446 33.94561)


In [26]:
# check unique ids
nsi_gdf[['fd_id','bid']].astype(str).describe()

,fd_id,bid
count,40566,40566
unique,40566,37826
top,508520377,865RWGRM+GPV-10-11-9-11
freq,1,11


In [27]:
# print columns
nsi_gdf.columns

Index(['fd_id', 'bid', 'occtype', 'st_damcat', 'bldgtype', 'found_type',
       'cbfips', 'pop2amu65', 'pop2amo65', 'pop2pmu65', 'pop2pmo65', 'sqft',
       'num_story', 'ftprntid', 'ftprntsrc', 'students', 'found_ht',
       'val_struct', 'val_cont', 'val_vehic', 'source', 'med_yr_blt',
       'firmzone', 'o65disable', 'u65disable', 'x', 'y', 'ground_elv',
       'ground_elv_m', 'geometry'],
      dtype='object')

In [28]:
# describe sqft
nsi_gdf['sqft'].describe()

count     40566.000000
mean       6531.229923
std       33554.939430
min         105.000000
25%        1264.000000
50%        1743.000000
75%        2560.000000
max      946400.000000
Name: sqft, dtype: float64

In [29]:
# how many buildings have missing sqft?
nsi_gdf['sqft'].isna().sum()

0

In [30]:
# group by occtype and count
nsi_gdf[['fd_id','occtype']].groupby('occtype').count()

,fd_id
occtype,
AGR1,79
COM1,572
COM10,15
COM2,381
COM3,474
COM4,1353
COM5,148
COM6,21
COM7,302


In [31]:
# prepare file for HUA
# add column for guid based on fd_id
nsi_gdf['guid'] = nsi_gdf['fd_id'].astype(str)+'-'+nsi_gdf['bid'].astype(str)
# add archetype based on occtype
nsi_gdf['archetype'] = nsi_gdf['occtype'].astype(str)
# add source column
nsi_gdf['bldgsource'] = 'NSI2023-05-09'
# Make new dataframe with key columns
nsi_gdf_hua = nsi_gdf[['guid','archetype','bldgsource','sqft','geometry']]
nsi_gdf_hua.head()

,guid,archetype,bldgsource,sqft,geometry
0,508520377-865RXJGC+98F-4-4-4-3,RES1-2SNB,NSI2023-05-09,4819.44000,POINT (-83.37919 33.97594)
1,508490030-865RWGVJ+GXV-48-76-38-78,COM4,NSI2023-05-09,499171.00000,POINT (-83.46758 33.94386)
2,508490031-865RWGWP+66V-4-3-4-3,REL1,NSI2023-05-09,1398.05333,POINT (-83.46446 33.94561)
3,508490032-865RWGWP+66V-4-3-4-3,COM7,NSI2023-05-09,4194.16000,POINT (-83.46446 33.94561)
4,508490033-865RWGWP+66V-4-3-4-3,COM4,NSI2023-05-09,2621.35000,POINT (-83.46446 33.94561)


In [32]:
# Check unique guid
nsi_gdf_hua['guid'].describe()

count                              40566
unique                             40566
top       508520377-865RXJGC+98F-4-4-4-3
freq                                   1
Name: guid, dtype: object

In [33]:
# Save file in my WorkNPR folder
folderpath = "../../../WorkNPR/SourceData/"
filename = f"nsi_01av1_hua_{county_fips}.shp"
nsi_gdf_hua.to_file(folderpath + filename)